In [ ]:
import pandas as pd
import keras
from keras.layers import Input, Dense, LSTM, Conv1D, \
    BatchNormalization, Dropout, MaxPooling1D, Flatten
import tensorflow as tf
import matplotlib.pyplot as plt
from Indicator import *
from DataScaler import *
from tensorflow.keras.utils import to_categorical

In [ ]:
data = pd.read_csv("labeled_data_120.csv").drop(["Unnamed: 0"],axis=1)
data.head()

In [ ]:
data = data[data.label != 2].reset_index(drop=True)

In [ ]:
def checking_labeling(labeling_data):
    step = 1000
    for i in range(0, len(labeling_data), step):
        name = str(labeling_data.iloc[i]["datetime"])+ " ~ " + str(labeling_data.iloc[i + step - 1]["datetime"])
        fig = plt.figure(figsize=(16,9))
        fig = plt.title(name)
        fig = plt.plot(labeling_data.iloc[i:i + step]["datetime"], labeling_data.iloc[i:i + step]["close"])
        for j in range(i, i + step):
            if labeling_data.iloc[j]["label"] == 0:
                fig = plt.scatter(labeling_data.iloc[j]["datetime"], labeling_data.iloc[j]["close"], color = "r")
            if labeling_data.iloc[j]["label"] == 1:
                fig = plt.scatter(labeling_data.iloc[j]["datetime"], labeling_data.iloc[j]["close"], color = "b")
        
        plt.savefig("Labeling_picture/" + name + ".png")

In [ ]:
parameter = [
        {"rsi" : {"period" : 14}},
        {"ma" : {"period" : 7}},
        {"ma" : {"period" : 25}},
        {"ema" :{"period" : 7}},
        {"ema" :{"period" : 25}},
        {"stochastic" : {"n" : 14,"m" : 5,"t" : 5}},
        {"bb" : {"length" : 21,"std" : 2}},
        {"kdj" : {}},
        {"macd" : {"fast_period": 12, "slow_period" : 26}}
]
DataManageBot = DataManage(data, parameter = parameter)
data = DataManageBot.get_data()
data.head()

In [ ]:
# test_size = int(len(df) * 0.2)
test_size = 1440 * 30
X,Y = data.drop(['label','datetime'],axis = 1),data['label']
X = Data_StandardScaler(X)

x_train = X[:-test_size]
y_train = Y[:-test_size]
x_test = X[-test_size:]
y_test = Y[-test_size:]
x_train.shape,y_train.shape

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(256,input_shape = (x_train.shape[1],)))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(128,activation="relu"))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(64,activation="relu"))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(32,activation="relu"))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(1,activation="sigmoid"))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                    metrics=[tf.keras.metrics.BinaryAccuracy()],
                    loss = tf.keras.losses.BinaryCrossentropy())

In [ ]:
model.fit(x_train,y_train,epochs=10)

In [ ]:
pred = model.predict(x_test)
pred[:10]

In [ ]:
data = pd.read_csv("testlabel.csv").drop(["Unnamed: 0"],axis=1)
df = data
df = add_rsi(df)
df = add_ma(df,period=7)
df = add_ema(df,period=7)
df = add_ma(df,period=25)
df = add_ema(df,period=25)
df = add_ma(df,period=99)
df = add_ema(df,period=99)
df = add_stochastic(df)
df = add_bb(df,length=21)
df = add_disparity(df,period=25)
df = add_macd(df)
df = add_kdj(df)
df = df.dropna()
df
x = df.drop(['datetime','label'],axis=1)
y = df.label
# x,y = make_dataset(x,y)
print(x.shape, y.shape)
# test_size = int(len(df) * 0.2)
test_size = 1440 * 30
X,Y = df.drop(['label','datetime'],axis = 1),df['label']
X = Data_StandardScaler(X)

x_train = X[:-test_size]
y_train = Y[:-test_size]
x_test = X[-test_size:]
y_test = Y[-test_size:]
x_train.shape,y_train.shape

In [ ]:
pred = model.predict(x_test)
pred[:10]

In [ ]:
result_label = []
cnt = 0
for i in range(len(pred)):
    if pred[i][0] > 0.9:
        result_label.append(1)
    elif pred[i][0] < 0.1:
        result_label.append(0)
    else:
        result_label.append(-1)
        cnt += 1
print(cnt)

In [ ]:
ck = 0 # 정답 카운트
cnt = 0 # 총 카운트
report_list = []
report_pred = []
for i in range(len(result_label)):
    if result_label[i] != -1:
        if result_label[i] == y_test.iloc[i]:
            ck += 1
        cnt += 1
print("accuracy :",round(ck/cnt * 100,2),"%")

In [ ]:
Data = df.iloc[-test_size:].drop(["datetime", "label"],axis = 1)
Data

In [ ]:
import matplotlib.pyplot as plt
def checking_labeling(labeling_data):
    step = 1000
    for i in range(0, len(labeling_data), step):
        name = str(labeling_data.iloc[i]["datetime"])+ " ~ " + str(labeling_data.iloc[i + step - 1]["datetime"])
        fig = plt.figure(figsize=(16,9))
        fig = plt.title(name)
        fig = plt.plot(labeling_data.iloc[i:i + step]["datetime"], labeling_data.iloc[i:i + step]["close"])
        for j in range(i, i + step):
            if labeling_data.iloc[j]["result_label"] == 0:
                fig = plt.scatter(labeling_data.iloc[j]["datetime"], labeling_data.iloc[j]["close"], color = "r")
            if labeling_data.iloc[j]["result_label"] == 1:
                fig = plt.scatter(labeling_data.iloc[j]["datetime"], labeling_data.iloc[j]["close"], color = "b")
        
        plt.savefig("Test_Result/" + name + ".png")

In [ ]:
result_data = pd.concat([df.iloc[-test_size:].reset_index(), pd.DataFrame(result_label,columns=["result_label"])],axis=1)
result_data

In [ ]:
checking_labeling(result_data)

In [1]:
X,Y = df.drop(['label','datetime'],axis = 1),df['label']
test_size = 1440*30
x_train = X[:-test_size]
y_train = Y[:-test_size]
x_test = X[-test_size:]
y_test = Y[-test_size:]
x_test['label'] = result_label
from tqdm import tqdm
set_amount = 0.002
time_window = 1
goal = 0.0015
fee = 0.0008
max_buy = 100000
for k in range(0,1):
    buy_count = 0
    buy_price = []
    result = 0
    result_list = []
    total = 0
    win = 0
    buy_count_list = []
    amount = 0.000
    average_risk = []
    average_price = 0
    trading = []
    buy_index = []
    for i in tqdm(range(10,len(x_test)-1)):
        if x_test.iloc[i]['label'] == 0 and buy_count < max_buy:
            buy_count += 1
            average_price = ( average_price * amount + x_test.iloc[i]['close'] * set_amount) / (amount + set_amount)
            amount += set_amount
            buy_price.append(x_test.iloc[i]['close'])
            buy_index.append(i)
        elif x_test.iloc[i-1]['label'] == 1 and x_test.iloc[i]['label'] == -1 and buy_count != 0:
            temp = (x_test.iloc[i]['close']-average_price) * amount - (average_price * fee * amount)
            line = [temp,x_test.iloc[i]['close'],average_price,amount,i]
            buy_count_list.append(buy_count)
            if temp > 0:
                win += 1
            average_price = 0
            total += 1
            result += temp
            buy_count = 0
            amount = 0.000
            buy_price.clear()
            trading.append(line)
        elif buy_count >= max_buy:
            temp = (x_test.iloc[i]['close'] - average_price) * amount - (average_price * 0.0008 * amount)
            line = [temp,x_test.iloc[i]['close'],average_price,amount,i]
            buy_count_list.append(buy_count)
            if temp > 0:
                win += 1
            average_price = 0
            total += 1
            result += temp
            buy_count = 0
            amount = 0.000
            buy_price.clear()
            trading.append(line)
        if buy_count > 0:
            average_risk.append(round((sum(buy_price)/buy_count-x_test.iloc[i]['close'])/x_test.iloc[i]['close'],2))
        result_list.append(result)
    buy_count_list.sort()
    average_risk.sort()
    print("k:",k)
    try:
        print('평균 매매 횟수',sum(buy_count_list)/len(buy_count_list))
        print('이익',result)
        print('승률',win/total)
        print("리스크",average_risk[-1])
        print("최대 보유",buy_count_list[-1])
        print("트레이딩 횟수:",total)
    except:
        print("ZeroDivisionError ")

NameError: name 'df' is not defined

In [ ]:
x_test.label.value_counts()